EXTRACT-TRANSFORM-LOAD approach 

In [1]:
import requests # allows API calls
import json # work with text in json format
import polars as pl # faster panda
import os
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi


In [2]:
# Load env variables from .env file
load_dotenv()

# get specific key from env
api_key = os.getenv('YT_KEY')

# get channel ID (Kurzgesagt)
channel_id='UCsXVk37bltHxD1rDPwtNM8Q'

# define url for API
url = 'https://www.googleapis.com/youtube/v3/search'

# init page token
page_token = None

# init list to store video data
video_record_list = []

In [3]:
def getVideoRecords(response: requests.models.Response) -> list:
    """
    Function to excract Youtube video data from get request response.
    """
    # creates a list of dictionaries, where each item in the list corresponds to one video

    # initialize list to store video data from page results
    video_record_list = [] 

    for raw_item in json.loads(response.text)['items']:

        # only extract youtube videos
        if raw_item['id']['kind'] != "youtube#video":
            continue # start again
        
        # extract video data and save it into dict
        video_record = {}
        video_record['video_id'] = raw_item['id']['videoId']
        video_record['datetime'] = raw_item['snippet']['publishedAt']
        video_record['title'] = raw_item['snippet']['title']

        # append that dictionary to video record list

        video_record_list.append(video_record)

    return video_record_list


def extract_text(transcript:list)->str:
    """
    function to extract text from transcript dict
    """
    text_list = [transcript[i]['text'] for i in range(len(transcript))]
    return ' '.join(text_list)

        

In [4]:
while page_token != 0:
    # define parameters for API call
    params = {'key': api_key, 'channelId': channel_id, 'part':["snippet","id"], 'order':"date", 'maxResults':50, 'pageToken':page_token}

    # make get request
    response = requests.get(url,params=params)

    # append video data from page results to list
    video_record_list += getVideoRecords(response)

    try:
        # grab next page token
        page_token = json.loads(response.text)['nextPageToken']

    except:
        page_token = 0

In [7]:
df = pl.DataFrame(video_record_list)
print(df)

shape: (259, 3)
┌─────────────┬──────────────────────┬─────────────────────────────────┐
│ video_id    ┆ datetime             ┆ title                           │
│ ---         ┆ ---                  ┆ ---                             │
│ str         ┆ str                  ┆ str                             │
╞═════════════╪══════════════════════╪═════════════════════════════════╡
│ Ce9kHZQk3MI ┆ 2025-06-09T17:00:14Z ┆ Test Our PC Game – Star Birds … │
│ zxq60I5RSW8 ┆ 2025-06-03T14:00:05Z ┆ How Nuclear Flies Protect You … │
│ o7aXV_PV4ss ┆ 2025-05-29T14:01:14Z ┆ Dozens of Nukes Are Missing. W… │
│ m6KnVTYtSc0 ┆ 2025-05-20T14:00:01Z ┆ Why Does Fentanyl Feel So Good… │
│ kYOkrswU1KE ┆ 2025-05-15T14:01:09Z ┆ Panic Attacks Explained in 60 … │
│ …           ┆ …                    ┆ …                               │
│ F3QpgXBtDeo ┆ 2013-11-28T17:03:32Z ┆ How The Stock Exchange Works (… │
│ UuGrBhK2c7U ┆ 2013-10-11T19:11:39Z ┆ The Gulf Stream Explained       │
│ Uti2niW2BRA ┆ 2013-09-03T09:12:24

In [8]:
# init list to store video captions 
transcript_text_list = []

# loop through each row of dataframe

for i in range(len(df)):

    try:
        # get the transcript
        transcript = YouTubeTranscriptApi.get_transcript(df['video_id'][i])
        # extract text transcript
        transcript_text = extract_text(transcript)
        
    
    # exception if captions are not available do not save (usually shorts)
    except: 
        transcript_text = "n/a"

    # append transcript text to list
    transcript_text_list.append(transcript_text)

print(transcript_text_list)


["Let's build a space factory. Starbirds is a relaxing base building game set in space. Help a flock of brave interstellar birds explore and mine countless asteroids to build thriving production networks. Unlock powerful technologies, travel to distant star systems, and advance further into space than anyone before. The demo is live on Steam right now. Jump into the story campaign. Start mining and keep things running smoothly while making use of whatever the galaxy decides to throw at you and your crew. We're still building Starbirds and all your feedback helps us make the game even better for the early access launch later this year. Head to Steam through the link in the description, get the demo, and start your mission into space. Thank you so much for your support so far. We truly couldn't have done this without", 'Right now 100 million radiation-blasted\xa0\n flesh-eating flies are raining down over the jungles of Panama – to commit \n a genocide that can never end. Their\xa0tiny b

In [9]:
# add transcript to dataframe
df = df.with_columns(pl.Series(name="transcript", values = transcript_text_list))
print(df.head())

shape: (5, 4)
┌─────────────┬──────────────────────┬──────────────────────────────┬──────────────────────────────┐
│ video_id    ┆ datetime             ┆ title                        ┆ transcript                   │
│ ---         ┆ ---                  ┆ ---                          ┆ ---                          │
│ str         ┆ str                  ┆ str                          ┆ str                          │
╞═════════════╪══════════════════════╪══════════════════════════════╪══════════════════════════════╡
│ Ce9kHZQk3MI ┆ 2025-06-09T17:00:14Z ┆ Test Our PC Game – Star      ┆ Let's build a space factory. │
│             ┆                      ┆ Birds …                      ┆ S…                           │
│ zxq60I5RSW8 ┆ 2025-06-03T14:00:05Z ┆ How Nuclear Flies Protect    ┆ Right now 100 million        │
│             ┆                      ┆ You …                        ┆ radiatio…                    │
│ o7aXV_PV4ss ┆ 2025-05-29T14:01:14Z ┆ Dozens of Nukes Are Missing. ┆ n/a    

In [10]:
# TRANSFROM
# check for dupliactes
print("shape: ", df.shape)
print("n unique rows: ", df.n_unique())
for j in range(df.shape[1]):
    print('n unique elements(' + df.columns[j] + ")", df[:,j].n_unique())

shape:  (259, 4)
n unique rows:  256
n unique elements(video_id) 256
n unique elements(datetime) 256
n unique elements(title) 255
n unique elements(transcript) 218


In [11]:
df = df.filter(df['transcript'] != 'n/a')
print("shape: ", df.shape)
print("n unique rows: ", df.n_unique())
for j in range(df.shape[1]):
    print('n unique elements(' + df.columns[j] + ")", df[:,j].n_unique())


shape:  (220, 4)
n unique rows:  217
n unique elements(video_id) 217
n unique elements(datetime) 217
n unique elements(title) 217
n unique elements(transcript) 217


In [12]:
duplicates = df.filter(~df.is_unique())
print(duplicates)


shape: (6, 4)
┌─────────────┬──────────────────────┬──────────────────────────┬──────────────────────────────────┐
│ video_id    ┆ datetime             ┆ title                    ┆ transcript                       │
│ ---         ┆ ---                  ┆ ---                      ┆ ---                              │
│ str         ┆ str                  ┆ str                      ┆ str                              │
╞═════════════╪══════════════════════╪══════════════════════════╪══════════════════════════════════╡
│ LBudghsdByQ ┆ 2023-10-04T14:00:02Z ┆ Why Humans Are Vanishing ┆ Every two years one million Ja…  │
│ BxXzzAEEhCA ┆ 2023-09-21T14:00:08Z ┆ Would You Choose Life On ┆ spending your entire life in s…  │
│             ┆                      ┆ Earth…                   ┆                                  │
│ LBudghsdByQ ┆ 2023-10-04T14:00:02Z ┆ Why Humans Are Vanishing ┆ Every two years one million Ja…  │
│ BxXzzAEEhCA ┆ 2023-09-21T14:00:08Z ┆ Would You Choose Life On ┆ spending yo

In [20]:
df = df.unique()
print(df.shape)

df = df.with_columns(pl.col('datetime').cast(pl.Datetime))
print(df.head())

(217, 4)
shape: (5, 4)
┌─────────────┬─────────────────────┬───────────────────────────────┬──────────────────────────────┐
│ video_id    ┆ datetime            ┆ title                         ┆ transcript                   │
│ ---         ┆ ---                 ┆ ---                           ┆ ---                          │
│ str         ┆ datetime[μs]        ┆ str                           ┆ str                          │
╞═════════════╪═════════════════════╪═══════════════════════════════╪══════════════════════════════╡
│ wwSzpaTHyS8 ┆ 2024-01-30 15:00:03 ┆ Did The Future Already        ┆ Do your past, present and    │
│             ┆                     ┆ Happen?…                      ┆ futu…                        │
│ dFCbJmgeHmA ┆ 2021-06-15 14:09:03 ┆ The Day the Dinosaurs Died –  ┆ one of the greatest          │
│             ┆                     ┆ M…                            ┆ Illusions …                  │
│ 1AElONvi9WQ ┆ 2020-03-01 13:30:00 ┆ Why Blue Whales Don't Get     

In [21]:
special_strings = ['&#39;', '&amp;']
replacements = ["'","&"]

# replace each special string appearing in title and transcript columns
for i in range(len(special_strings)):
    df = df.with_columns(df['title'].str.replace(special_strings[i], replacements[i]).alias('title'))
    df = df.with_columns(df['transcript'].str.replace(special_strings[i], replacements[i]).alias('transcript'))

In [22]:
df.write_parquet('data/video-transcripts.parquet')

In [24]:
df = df.with_columns(pl.col('datetime').cast(pl.Datetime))
df.write_csv('data/video-transcripts.csv')

This file makes a complete data pipeline:

Data Source(YouTube) -> Data Pipeline (ETL) -> Data Store (video-transcript.parquet)

with Data corrections